# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-04-27 12:20:46] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=35250, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=710605549, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None

[2025-04-27 12:20:57 TP0] Attention backend not set. Use fa3 backend by default.
[2025-04-27 12:20:57 TP0] Init torch distributed begin.


[2025-04-27 12:20:58 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-27 12:20:58 TP0] Load weight begin. avail mem=77.57 GB


[2025-04-27 12:20:58 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-27 12:21:00 TP0] Using model weights format ['*.safetensors']
[2025-04-27 12:21:00 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

[2025-04-27 12:21:01 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768


[2025-04-27 12:21:02] INFO:     Started server process [1586933]
[2025-04-27 12:21:02] INFO:     Waiting for application startup.
[2025-04-27 12:21:02] INFO:     Application startup complete.
[2025-04-27 12:21:02] INFO:     Uvicorn running on http://0.0.0.0:35250 (Press CTRL+C to quit)


[2025-04-27 12:21:02] INFO:     127.0.0.1:45180 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-27 12:21:03] INFO:     127.0.0.1:45184 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-27 12:21:03 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-04-27 12:21:07] INFO:     127.0.0.1:45190 - "POST /generate HTTP/1.1" 200 OK
[2025-04-27 12:21:07] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-04-27 12:21:07 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-04-27 12:21:08 TP0] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, gen throughput (token/s): 6.20, #queue-req: 0


[2025-04-27 12:21:08 TP0] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, gen throughput (token/s): 132.08, #queue-req: 0


[2025-04-27 12:21:08 TP0] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, gen throughput (token/s): 135.40, #queue-req: 0
[2025-04-27 12:21:08] INFO:     127.0.0.1:50042 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-04-27 12:21:08] INFO:     127.0.0.1:50048 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-04-27 12:21:08] INFO:     127.0.0.1:50060 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-04-27 12:21:08 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-04-27 12:21:09] INFO:     127.0.0.1:50062 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-04-27 12:21:09] INFO:     127.0.0.1:50072 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-04-27 12:21:09 TP0] Cache flushed successfully!
[2025-04-27 12:21:09] INFO:     127.0.0.1:50080 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-04-27 12:21:09] Start update_weights. Load format=auto
[2025-04-27 12:21:09 TP0] Update engine weights online from disk begin. avail mem=41.66 GB
[2025-04-27 12:21:09 TP0] Using model weights format ['*.safetensors']


[2025-04-27 12:21:10 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.69it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.68it/s]

[2025-04-27 12:21:10 TP0] Update weights end.
[2025-04-27 12:21:10 TP0] Cache flushed successfully!
[2025-04-27 12:21:10] INFO:     127.0.0.1:50096 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-04-27 12:21:10] Start update_weights. Load format=auto
[2025-04-27 12:21:10 TP0] Update engine weights online from disk begin. avail mem=41.66 GB
[2025-04-27 12:21:10 TP0] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-04-27 12:21:10] INFO:     127.0.0.1:50104 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-27 12:21:19] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-1.5B-instruct', chat_template=None, completion_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=37763, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=779008459, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='

[2025-04-27 12:21:28 TP0] Downcasting torch.float32 to torch.float16.


[2025-04-27 12:21:28 TP0] Overlap scheduler is disabled for embedding models.
[2025-04-27 12:21:28 TP0] Downcasting torch.float32 to torch.float16.
[2025-04-27 12:21:28 TP0] Attention backend not set. Use fa3 backend by default.
[2025-04-27 12:21:28 TP0] Init torch distributed begin.


[2025-04-27 12:21:29 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-27 12:21:29 TP0] Load weight begin. avail mem=78.58 GB


[2025-04-27 12:21:30 TP0] Ignore import error when loading sglang.srt.models.llama4. 


...

[2025-04-27 12:21:34 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.27 GB, V size: 0.27 GB
[2025-04-27 12:21:34 TP0] Memory pool end. avail mem=63.85 GB


[2025-04-27 12:21:35 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-27 12:21:35] INFO:     Started server process [1589869]
[2025-04-27 12:21:35] INFO:     Waiting for application startup.
[2025-04-27 12:21:35] INFO:     Application startup complete.
[2025-04-27 12:21:35] INFO:     Uvicorn running on http://0.0.0.0:37763 (Press CTRL+C to quit)
[2025-04-27 12:21:35] INFO:     127.0.0.1:40818 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-27 12:21:36] INFO:     127.0.0.1:40834 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-27 12:21:36 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-04-27 12:21:38] INFO:     127.0.0.1:40842 - "POST /encode HTTP/1.1" 200 OK
[2025-04-27 12:21:38] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-04-27 12:21:40 TP0] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-04-27 12:21:40] INFO:     127.0.0.1:40854 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [14]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-27 12:21:49] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=32468, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=904555460, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step

[2025-04-27 12:21:58 TP0] Overlap scheduler is disabled for embedding models.
[2025-04-27 12:21:58 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-27 12:21:58 TP0] Init torch distributed begin.


[2025-04-27 12:21:59 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-27 12:21:59 TP0] Load weight begin. avail mem=78.58 GB
[2025-04-27 12:21:59 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-27 12:21:59 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.08it/s]


...

[2025-04-27 12:22:03 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-27 12:22:04] INFO:     Started server process [1592596]
[2025-04-27 12:22:04] INFO:     Waiting for application startup.
[2025-04-27 12:22:04] INFO:     Application startup complete.
[2025-04-27 12:22:04] INFO:     Uvicorn running on http://0.0.0.0:32468 (Press CTRL+C to quit)


[2025-04-27 12:22:04] INFO:     127.0.0.1:53634 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-27 12:22:05] INFO:     127.0.0.1:53646 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-27 12:22:05 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-04-27 12:22:05,936 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90


In [15]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

2025-04-27 12:22:45,745 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-04-27 12:22:45,756 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-04-27 12:23:06,441 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-04-27 12:23:06 TP0] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-04-27 12:23:06] INFO:     127.0.0.1:53662 - "POST /encode HTTP/1.1" 200 OK
[2025-04-27 12:23:06] The server is fired up and ready to roll!
2025-04-27 12:23:06,668 - INFO - flashinfer.jit: Loading JIT ops: cascade


2025-04-27 12:23:20,976 - INFO - flashinfer.jit: Finished loading JIT ops: cascade
[2025-04-27 12:23:20] INFO:     127.0.0.1:33348 - "POST /classify HTTP/1.1" 200 OK


In [16]:
terminate_process(reward_process)

[2025-04-27 12:23:21] Child process unexpectedly failed with an exit code 9. pid=1593034
[2025-04-27 12:23:21] Child process unexpectedly failed with an exit code 9. pid=1592965


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [17]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-04-27 12:23:29] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=33857, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=547546873, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_reques

[2025-04-27 12:23:39 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-27 12:23:39 TP0] Init torch distributed begin.


[2025-04-27 12:23:40 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-27 12:23:40 TP0] Load weight begin. avail mem=78.08 GB
[2025-04-27 12:23:40 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-27 12:23:40 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  12% Completed | 1/8 [00:00<00:04,  1.42it/s]


...

[2025-04-27 12:23:48] INFO:     127.0.0.1:50554 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-27 12:23:48 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-04-27 12:23:51,354 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90


2025-04-27 12:23:51,625 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-04-27 12:23:51,634 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-04-27 12:23:51,787 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-04-27 12:23:51 TP0] Using default MoE config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1b-gpu-45/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can tune the config with https://github.com/sgl-project/sglang/blob/main/benchmark/kernels/fused_moe_triton/tuning_fused_moe_triton.py.


2025-04-27 12:23:52,422 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


In [18]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

import glob

output_file = glob.glob("expert_distribution_*.csv")[0]
with open(output_file, "r") as f:
    print_highlight("\n| Layer ID | Expert ID | Count |")
    print_highlight("|----------|-----------|--------|")
    next(f)
    for i, line in enumerate(f):
        if i < 9:
            layer_id, expert_id, count = line.strip().split(",")
            print_highlight(f"| {layer_id:8} | {expert_id:9} | {count:6} |")

2025-04-27 12:24:12,036 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False
[2025-04-27 12:24:12 TP0] Resetting expert distribution record...
[2025-04-27 12:24:12] INFO:     127.0.0.1:50570 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-04-27 12:24:12 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-04-27 12:24:12] INFO:     127.0.0.1:50566 - "POST /generate HTTP/1.1" 200 OK
[2025-04-27 12:24:12] The server is fired up and ready to roll!


[2025-04-27 12:24:13 TP0] Decode batch. #running-req: 1, #token: 45, token usage: 0.00, gen throughput (token/s): 1.72, #queue-req: 0


[2025-04-27 12:24:14 TP0] Decode batch. #running-req: 1, #token: 85, token usage: 0.00, gen throughput (token/s): 44.93, #queue-req: 0


[2025-04-27 12:24:14 TP0] Decode batch. #running-req: 1, #token: 125, token usage: 0.01, gen throughput (token/s): 44.73, #queue-req: 0


[2025-04-27 12:24:15] INFO:     127.0.0.1:54770 - "POST /generate HTTP/1.1" 200 OK


[2025-04-27 12:24:15] INFO:     127.0.0.1:54784 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-04-27 12:24:15 TP0] Resetting expert distribution record...
[2025-04-27 12:24:15] INFO:     127.0.0.1:54786 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [19]:
terminate_process(expert_record_server_process)

[2025-04-27 12:24:15] Child process unexpectedly failed with an exit code 9. pid=1598608
[2025-04-27 12:24:15] Child process unexpectedly failed with an exit code 9. pid=1598542


## Skip Tokenizer and Detokenizer

SGLang Runtime also supports skip tokenizer and detokenizer. This is useful in cases like integrating with RLHF workflow.

In [20]:
tokenizer_free_server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --skip-tokenizer-init
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-27 12:24:26] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=True, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=33503, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=161464129, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=Non

[2025-04-27 12:24:37 TP0] Attention backend not set. Use fa3 backend by default.
[2025-04-27 12:24:37 TP0] Init torch distributed begin.


[2025-04-27 12:24:37 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-27 12:24:37 TP0] Load weight begin. avail mem=62.17 GB


[2025-04-27 12:24:37 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-27 12:24:38 TP0] Using model weights format ['*.safetensors']


[2025-04-27 12:24:38 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.71it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.71it/s]



[2025-04-27 12:24:39 TP0] Load weight end. type=Qwen2ForCausalLM, dtype=torch.bfloat16, avail mem=47.84 GB, mem usage=14.33 GB.
[2025-04-27 12:24:39 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.12 GB, V size: 0.12 GB
[2025-04-27 12:24:39 TP0] Memory pool end. avail mem=47.43 GB
[2025-04-27 12:24:39 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768
[2025-04-27 12:24:39] INFO:     Started server process [1601619]
[2025-04-27 12:24:39] INFO:     Waiting for application startup.
[2025-04-27 12:24:39] INFO:     Application startup complete.
[2025-04-27 12:24:39] INFO:     Uvicorn running on http://127.0.0.1:33503 (Press CTRL+C to quit)


[2025-04-27 12:24:39] INFO:     127.0.0.1:44896 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-27 12:24:40] INFO:     127.0.0.1:44902 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-27 12:24:40 TP0] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-04-27 12:24:43] INFO:     127.0.0.1:44910 - "POST /generate HTTP/1.1" 200 OK
[2025-04-27 12:24:43] The server is fired up and ready to roll!


In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("qwen/qwen2.5-0.5b-instruct")

input_text = "What is the capital of France?"

input_tokens = tokenizer.encode(input_text)
print_highlight(f"Input Text: {input_text}")
print_highlight(f"Tokenized Input: {input_tokens}")

response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "input_ids": input_tokens,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 256,
            "stop_token_ids": [tokenizer.eos_token_id],
        },
        "stream": False,
    },
)
output = response.json()
output_tokens = output["output_ids"]

output_text = tokenizer.decode(output_tokens, skip_special_tokens=False)
print_highlight(f"Tokenized Output: {output_tokens}")
print_highlight(f"Decoded Output: {output_text}")
print_highlight(f"Output Text: {output['meta_info']['finish_reason']}")

[2025-04-27 12:24:44 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-04-27 12:24:45 TP0] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, gen throughput (token/s): 6.84, #queue-req: 0


[2025-04-27 12:24:45 TP0] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, gen throughput (token/s): 146.10, #queue-req: 0


[2025-04-27 12:24:45 TP0] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, gen throughput (token/s): 135.78, #queue-req: 0


[2025-04-27 12:24:45 TP0] Decode batch. #running-req: 1, #token: 160, token usage: 0.01, gen throughput (token/s): 136.45, #queue-req: 0


[2025-04-27 12:24:46 TP0] Decode batch. #running-req: 1, #token: 200, token usage: 0.01, gen throughput (token/s): 136.25, #queue-req: 0


[2025-04-27 12:24:46 TP0] Decode batch. #running-req: 1, #token: 240, token usage: 0.01, gen throughput (token/s): 127.61, #queue-req: 0
[2025-04-27 12:24:46] INFO:     127.0.0.1:44926 - "POST /generate HTTP/1.1" 200 OK


In [22]:
terminate_process(tokenizer_free_server_process)

[2025-04-27 12:24:46] Child process unexpectedly failed with an exit code 9. pid=1602423
[2025-04-27 12:24:46] Child process unexpectedly failed with an exit code 9. pid=1602291
